<a href="https://colab.research.google.com/github/tyri0n11/distributed-system/blob/main/7_1_classification_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binary Classification


# Create Spark entry points

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext(conf=SparkConf().setAppName("appName").setMaster("local[2]"))
spark = SparkSession(sparkContext=sc)

# Step 1: As usual, define the problem

**What sorts of people were likely to survive from the Titanic accident?**

# Step 2: gather the data

The datasets can be found here: [https://www.kaggle.com/c/titanic/data](https://www.kaggle.com/c/titanic/data).
It is also available in this github repository:
    
* [data/titanic/gender_submission.csv](data/titanic/gender_submission.csv)
* [data/titanic/test.csv](data/titanic/test.csv)
* [data/titanic/train.csv](data/titanic/train.csv)

# Step 3: prepare data for consumption

In [4]:
# load packages
import sys
print('Python version: {}'. format(sys.version))

import pandas as pd
print('Python version: {}'. format(pd.__version__))

import matplotlib
print('matplotlib version: {}'. format(matplotlib.__version__))

import numpy as np
print('numpy version: {}'. format(np.__version__))

import scipy as sp
print('scipy version: {}'. format(sp.__version__))

import IPython
from IPython import display # pretty printing of dataframe in Jupyter notebook
print('IPython version: {}'. format(IPython.__version__))

import pyspark
print('Apache Spark Pyspark version: {}'. format(pyspark.__version__)) # pyspark version

# misc libraries
import random
import time

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Python version: 2.2.2
matplotlib version: 3.10.0
numpy version: 2.0.2
scipy version: 1.16.3
IPython version: 7.34.0
Apache Spark Pyspark version: 4.0.1
-------------------------


In [5]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import OneVsRest

In [6]:
from subprocess import check_output
print('-'*10, 'Files', '-'*10)
print(check_output(['ls', './']).decode('utf8'))

---------- Files ----------
cuse_binary.csv
gender_submission.csv
sample_data
test.csv
train.csv



In [7]:
data_raw = spark.read.csv('./train.csv', inferSchema=True, header=True)

# the test file provided is for validation of final model.
data_val = spark.read.csv('./test.csv', inferSchema=True, header=True)

# preview the data
# data type
print('-'*10, 'data types', '-'*10)
pd.DataFrame(data_raw.dtypes)

---------- data types ----------


,0,1
0,PassengerId,int
1,Survived,int
2,Pclass,int
3,Name,string
4,Sex,string
5,Age,double
6,SibSp,int
7,Parch,int
8,Ticket,string
9,Fare,double


In [8]:
# data summary
print('-'*10, 'data summary', '-'*10)
data_raw.describe().toPandas()

---------- data summary ----------


,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889
1,mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
2,stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0.0,A10,C
4,max,891,1,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S


In [9]:
# view a small subset of the data
print('-'*10, 'randomely sample 1% data to view', '-'*10)
data_raw.randomSplit([0.01, 0.99])[0].toPandas()

---------- randomely sample 1% data to view ----------


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,112,0,3,"Zabour, Miss. Hileni",female,14.5,1,0,2665,14.4542,None,C
1,163,0,3,"Bengtsson, Mr. John Viktor",male,26.0,0,0,347068,7.7750,None,S
2,325,0,3,"Sage, Mr. George John Jr",male,NaN,8,2,CA. 2343,69.5500,None,S
3,357,1,1,"Bowerman, Miss. Elsie Edith",female,22.0,0,1,113505,55.0000,E33,S
4,453,0,1,"Foreman, Mr. Benjamin Laventall",male,30.0,0,0,113051,27.7500,C111,C
5,545,0,1,"Douglas, Mr. Walter Donald",male,50.0,1,0,PC 17761,106.4250,C86,C
6,624,0,3,"Hansen, Mr. Henry Damsgaard",male,21.0,0,0,350029,7.8542,None,S
7,762,0,3,"Nirva, Mr. Iisakki Antino Aijo",male,41.0,0,0,SOTON/O2 3101272,7.1250,None,S
8,794,0,1,"Hoyt, Mr. William Fisher",male,NaN,0,0,PC 17600,30.6958,None,C
9,819,0,3,"Holm, Mr. John Fredrik Alexander",male,43.0,0,0,C 7075,6.4500,None,S


### 3.21 The 4 C's of data cleaning: Correcting, Completing, Creating, and Converting

In [10]:
# we first check which values are NULL values for each column
# then we convert the boolean values to int (0 and 1), then we can count how many 1's exist in each column.
print('-'*25)
print('0: is not NULL')
print('1: is NULL')
print('-'*25)
print(' '*25)
# we build column strings and then use eval() to convert strings to column expressions.
data_raw.select([eval('data_raw.' + x + '.isNull().cast("int").alias("' + x + '")') for x in data_raw.columns]).show(n=10)

-------------------------
0: is not NULL
1: is NULL
-------------------------
                         
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    1|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    1|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    1|       0|
|          0|       0|     0|   0|  0|  1|    0|    0|     0|   0|    1|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    1|    

In [11]:
print('Train columns with null values:')
print('-'*25)
data_raw.select([eval('data_raw.' + x + '.isNull().cast("int").alias("' + x + '")') for x in data_raw.columns]).\
    groupBy().sum().toPandas()

Train columns with null values:
-------------------------


,sum(PassengerId),sum(Survived),sum(Pclass),sum(Name),sum(Sex),sum(Age),sum(SibSp),sum(Parch),sum(Ticket),sum(Fare),sum(Cabin),sum(Embarked)
0,0,0,0,0,0,177,0,0,0,0,687,2


In [12]:
print('Test columns with null values:')
print('-'*25)
data_val.select([eval('data_val.' + x + '.isNull().cast("int").alias("' + x + '")') for x in data_val.columns]).\
    groupBy().sum().toPandas()

Test columns with null values:
-------------------------


,sum(PassengerId),sum(Pclass),sum(Name),sum(Sex),sum(Age),sum(SibSp),sum(Parch),sum(Ticket),sum(Fare),sum(Cabin),sum(Embarked)
0,0,0,0,0,86,0,0,0,1,327,0


#### Above stuffs are normally things we need to prepare before jumping into any further steps of learning

### Exercise
1. Check and fill NA for NULL position using Spark.
2. Create new cleaned train and test data after filling NA, correcting datatype.

### Exercise
Do EDA with Titanic dataset using Spark.

In [13]:
df = spark.read.csv('./train.csv', inferSchema=True, header=True)
df.show(5, truncate=False)
df.printSchema()
df.describe().toPandas()

+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                               |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                            |male  |22.0|1    |0    |A/5 21171       |7.25   |NULL |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1    |0    |PC 17599        |71.2833|C85  |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                             |female|26.0|0    |0    |STON/O2. 3101282|7.925  |NULL |S       |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath (Lily May Peel)       |female|35.0|1    |0    |113803          |53

,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889
1,mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
2,stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0.0,A10,C
4,max,891,1,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S


In [14]:
from pyspark.sql.functions import col, isnan, count, when
from pyspark.sql.types import DoubleType, FloatType

exprs = []

for field in df.schema.fields:
    c = field.name
    dtype = field.dataType

    # Numeric → check NULL + NaN
    if isinstance(dtype, (DoubleType, FloatType)):
        exprs.append(
            count(
                when(col(c).isNull() | isnan(col(c)), c)
            ).alias(c)
        )
    # Non-numeric → chỉ check NULL
    else:
        exprs.append(
            count(
                when(col(c).isNull(), c)
            ).alias(c)
        )

missing_df = df.select(exprs)
missing_df.show(truncate=False)

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|0          |0       |0     |0   |0  |177|0    |0    |0     |0   |687  |2       |
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [15]:
cols_to_be_fill = ['Age', 'Cabin', 'Embarked']
df.select(*cols_to_be_fill).describe().show()

+-------+------------------+-----+--------+
|summary|               Age|Cabin|Embarked|
+-------+------------------+-----+--------+
|  count|               714|  204|     889|
|   mean| 29.69911764705882| NULL|    NULL|
| stddev|14.526497332334035| NULL|    NULL|
|    min|              0.42|  A10|       C|
|    max|              80.0|    T|       S|
+-------+------------------+-----+--------+



In [16]:
age_median = df.approxQuantile("Age", [0.5], 0.01)[0]
df = df.fillna({"Age": age_median})


In [17]:
df = df.fillna({"Cabin": "Unknown"})


In [18]:
from pyspark.sql.functions import col

embarked_mode = (
    df.groupBy("Embarked")
      .count()
      .orderBy(col("count").desc())
      .first()[0]
)

df = df.fillna({"Embarked": embarked_mode})


In [19]:
from pyspark.sql.functions import sum as spark_sum

df.select([
    spark_sum(col(c).isNull().cast("int")).alias(c)
    for c in cols_to_be_fill
]).show()


+---+-----+--------+
|Age|Cabin|Embarked|
+---+-----+--------+
|  0|    0|       0|
+---+-----+--------+



In [20]:
df.select(*cols_to_be_fill).describe().show()

+-------+------------------+-------+--------+
|summary|               Age|  Cabin|Embarked|
+-------+------------------+-------+--------+
|  count|               891|    891|     891|
|   mean| 29.36158249158249|   NULL|    NULL|
| stddev|13.019696550973201|   NULL|    NULL|
|    min|              0.42|    A10|       C|
|    max|              80.0|Unknown|       S|
+-------+------------------+-------+--------+



# Decision tree classification with pyspark

In [21]:
cuse = spark.read.csv('./cuse_binary.csv', header=True, inferSchema=True)
cuse.show(5)

+---+---------+---------+---+
|age|education|wantsMore|  y|
+---+---------+---------+---+
|<25|      low|      yes|  0|
|<25|      low|      yes|  0|
|<25|      low|      yes|  0|
|<25|      low|      yes|  0|
|<25|      low|      yes|  0|
+---+---------+---------+---+
only showing top 5 rows


## Process categorical columns
The following code does three things with pipeline:

* **`StringIndexer`** all categorical columns
* **`OneHotEncoder`** all categorical index columns
* **`VectorAssembler`** all feature columns into one vector column

In [22]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

# categorical columns
categorical_columns = cuse.columns[0:3]

#### Build StringIndexer stages

In [23]:
stringindexer_stages = [StringIndexer(inputCol=c, outputCol='strindexed_' + c) for c in categorical_columns]
# encode label column and add it to stringindexer_stages
stringindexer_stages += [StringIndexer(inputCol='y', outputCol='label')]

#### Build OneHotEncoder stages

In [24]:
onehotencoder_stages = [OneHotEncoder(inputCol='strindexed_' + c, outputCol='onehot_' + c) for c in categorical_columns]

#### Build VectorAssembler stage

In [25]:
feature_columns = ['onehot_' + c for c in categorical_columns]
vectorassembler_stage = VectorAssembler(inputCols=feature_columns, outputCol='features')

#### Build pipeline model

In [26]:
# all stages
all_stages = stringindexer_stages + onehotencoder_stages + [vectorassembler_stage]
pipeline = Pipeline(stages=all_stages)

In [27]:
all_stages

[StringIndexer_a811b65d02f5,
 StringIndexer_177b86722cf6,
 StringIndexer_24371033bd3d,
 StringIndexer_3c4d23f495f7,
 OneHotEncoder_1f08999fbb92,
 OneHotEncoder_2ec61bc66e54,
 OneHotEncoder_a7c209db32b7,
 VectorAssembler_8fd92dd2d326]

#### Fit pipeline model

In [28]:
pipeline_model = pipeline.fit(cuse)

#### Transform data

In [29]:
final_columns = feature_columns + ['features', 'label']
cuse_df = pipeline_model.transform(cuse).\
            select(final_columns)

cuse_df.show(5)

+-------------+----------------+----------------+-------------------+-----+
|   onehot_age|onehot_education|onehot_wantsMore|           features|label|
+-------------+----------------+----------------+-------------------+-----+
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
|(3,[2],[1.0])|       (1,[],[])|   (1,[0],[1.0])|(5,[2,4],[1.0,1.0])|  0.0|
+-------------+----------------+----------------+-------------------+-----+
only showing top 5 rows


### Split data into training and test datasets

In [30]:
training, test = cuse_df.randomSplit([0.8, 0.2], seed=1234)

### Build cross-validation model

#### Estimator

In [31]:
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol='features', labelCol='label')

#### Parameter grid

In [32]:
from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder().\
    addGrid(dt.maxDepth, [2,3,4,5]).\
    build()

#### Evaluator

In [33]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName="areaUnderROC")

#### Build cross-validation model

In [34]:
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=4)

In [35]:
type(cv)

pyspark.ml.tuning.CrossValidator

#### Fit cross-validation mode

In [36]:
cv_model = cv.fit(cuse_df)

#### Prediction

In [37]:
show_columns = ['features', 'label', 'prediction', 'rawPrediction', 'probability']

##### Prediction on training data

In [38]:
pred_training_cv = cv_model.transform(training)
pred_training_cv.select(show_columns).show(5, truncate=False)

+---------+-----+----------+-------------+----------------------------------------+
|features |label|prediction|rawPrediction|probability                             |
+---------+-----+----------+-------------+----------------------------------------+
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
+---------+-----+----------+-------------+----------------------------------------+
only showing top 5 rows


##### Prediction on test data

In [39]:
pred_test_cv = cv_model.transform(test)
pred_test_cv.select(show_columns).show(5, truncate=False)

+---------+-----+----------+-------------+----------------------------------------+
|features |label|prediction|rawPrediction|probability                             |
+---------+-----+----------+-------------+----------------------------------------+
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
|(5,[],[])|0.0  |1.0       |[203.0,237.0]|[0.46136363636363636,0.5386363636363637]|
+---------+-----+----------+-------------+----------------------------------------+
only showing top 5 rows


### Confusion matrix

Pyspark doesn’t have a function to calculate the confusion matrix automatically, but we can still easily get a confusion matrix with a combination use of several methods from the RDD class.

In [40]:
label_and_pred = cv_model.transform(cuse_df).select('label', 'prediction')
label_and_pred.rdd.zipWithIndex().countByKey()

defaultdict(int,
            {Row(label=0.0, prediction=0.0): 897,
             Row(label=0.0, prediction=1.0): 203,
             Row(label=1.0, prediction=0.0): 270,
             Row(label=1.0, prediction=1.0): 237})

### Parameters from the best model

In [41]:
print('The best MaxDepth is:', cv_model.bestModel._java_obj.getMaxDepth())

The best MaxDepth is: 3


# Gradient boost tree classification

In [42]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

gbt = GBTClassifier(featuresCol='features', labelCol='label')

param_grid = ParamGridBuilder().\
    addGrid(gbt.maxDepth, [2, 3, 4]).\
    addGrid(gbt.minInfoGain, [0.0, 0.1, 0.2, 0.3]).\
    addGrid(gbt.stepSize, [0.05, 0.1, 0.2, 0.4]).\
    build()

evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction')

In [43]:
from pyspark.ml.tuning import CrossValidator

crossvalidation = CrossValidator(estimator=gbt, estimatorParamMaps=param_grid, evaluator=evaluator)

In [44]:
crossvalidation_mod = crossvalidation.fit(cuse_df)

In [45]:
# In train
pred_train = crossvalidation_mod.transform(training)
pred_train.show(5)

+----------+----------------+----------------+---------+-----+--------------------+--------------------+----------+
|onehot_age|onehot_education|onehot_wantsMore| features|label|       rawPrediction|         probability|prediction|
+----------+----------------+----------------+---------+-----+--------------------+--------------------+----------+
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0177333722013...|[0.49113424322535...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0177333722013...|[0.49113424322535...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0177333722013...|[0.49113424322535...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0177333722013...|[0.49113424322535...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0177333722013...|[0.49113424322535...|       1.0|
+----------+----------------+----------------+---------+-----+----------

In [46]:
# In test
pred_test = crossvalidation_mod.transform(test)
pred_test.show(5)

+----------+----------------+----------------+---------+-----+--------------------+--------------------+----------+
|onehot_age|onehot_education|onehot_wantsMore| features|label|       rawPrediction|         probability|prediction|
+----------+----------------+----------------+---------+-----+--------------------+--------------------+----------+
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0177333722013...|[0.49113424322535...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0177333722013...|[0.49113424322535...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0177333722013...|[0.49113424322535...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0177333722013...|[0.49113424322535...|       1.0|
| (3,[],[])|       (1,[],[])|       (1,[],[])|(5,[],[])|  0.0|[-0.0177333722013...|[0.49113424322535...|       1.0|
+----------+----------------+----------------+---------+-----+----------

In [47]:
# Performance
print('Accuracy on training data (areaUnderROC): ', evaluator.setMetricName('areaUnderROC').evaluate(pred_train), "\n"
      'Accuracy on test data (areaUnderROC): ', evaluator.setMetricName('areaUnderROC').evaluate(pred_test))


Accuracy on training data (areaUnderROC):  0.632389174374786 
Accuracy on test data (areaUnderROC):  0.6811111111111111


In [48]:
# Confusion matrix in train
label_pred_train = pred_train.select('label', 'prediction')
label_pred_train.rdd.zipWithIndex().countByKey()

defaultdict(int,
            {Row(label=0.0, prediction=1.0): 167,
             Row(label=1.0, prediction=1.0): 190,
             Row(label=0.0, prediction=0.0): 708,
             Row(label=1.0, prediction=0.0): 227})

In [49]:
# Confusion matrix in test
label_pred_test = pred_test.select('label', 'prediction')
label_pred_test.rdd.zipWithIndex().countByKey()

defaultdict(int,
            {Row(label=0.0, prediction=1.0): 36,
             Row(label=1.0, prediction=1.0): 47,
             Row(label=0.0, prediction=0.0): 189,
             Row(label=1.0, prediction=0.0): 43})

In [50]:
# Best model and params
print('max depth: ', crossvalidation_mod.bestModel._java_obj.getMaxDepth(), "\n",
     'min information gain: ', crossvalidation_mod.bestModel._java_obj.getMinInfoGain(), "\n",
     'step size: ', crossvalidation_mod.bestModel._java_obj.getStepSize())

max depth:  2 
 min information gain:  0.0 
 step size:  0.05


### Exercise:
Do the classification in the case of cuse binary dataset above for other ML model in Spark.

```python
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import OneVsRest
```

In [51]:
lr = LogisticRegression(
    featuresCol="features",
    labelCol="label",
    maxIter=100,
    regParam=0.01,
    elasticNetParam=0.0
)

lr_model = lr.fit(training)
lr_pred = lr_model.transform(test)

In [52]:
dt = DecisionTreeClassifier(
    featuresCol="features",
    labelCol="label",
    maxDepth=6
)

dt_model = dt.fit(training)
dt_pred = dt_model.transform(test)


In [53]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    numTrees=100,
    maxDepth=8,
    seed=42
)

rf_model = rf.fit(training)
rf_pred = rf_model.transform(test)


In [54]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(
    featuresCol="features",
    labelCol="label",
    maxIter=50,
    maxDepth=5,
    stepSize=0.1
)

gbt_model = gbt.fit(training)
gbt_pred = gbt_model.transform(test)


In [55]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(
    featuresCol="features",
    labelCol="label",
    modelType="multinomial"
)

nb_model = nb.fit(training)
nb_pred = nb_model.transform(test)


In [56]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

num_features = training.select("features").first()[0].size

mlp = MultilayerPerceptronClassifier(
    featuresCol="features",
    labelCol="label",
    layers=[num_features, 32, 16, 2],
    maxIter=100,
    blockSize=128,
    seed=42
)

mlp_model = mlp.fit(training)
mlp_pred = mlp_model.transform(test)


In [57]:
from pyspark.ml.classification import OneVsRest, LinearSVC

svc = LinearSVC(
    featuresCol="features",
    labelCol="label",
    maxIter=100
)

ovr = OneVsRest(
    classifier=svc,
    featuresCol="features",
    labelCol="label"
)

ovr_model = ovr.fit(training)
ovr_pred = ovr_model.transform(test)


In [58]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

for name, pred in [
    ("LR", lr_pred),
    ("DT", dt_pred),
    ("RF", rf_pred),
    ("GBT", gbt_pred),
    ("NB", nb_pred),
    ("MLP", mlp_pred),
]:
    auc = evaluator.evaluate(pred)
    print(f"{name} AUC = {auc:.4f}")


LR AUC = 0.6715
DT AUC = 0.5815
RF AUC = 0.6846
GBT AUC = 0.6851
NB AUC = 0.6628
MLP AUC = 0.6621


## Exercise
Build decision tree and Gradient boosting classifier for the data Titanic above.


In [78]:
train_df = spark.read.csv('train.csv', header=True, inferSchema=True)
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)


In [79]:
from pyspark.sql.functions import col, isnan, count, when
from pyspark.sql.types import DoubleType, FloatType

exprs = []

for field in train_df.schema.fields:
    c = field.name
    dtype = field.dataType

    # Numeric → check NULL + NaN
    if isinstance(dtype, (DoubleType, FloatType)):
        exprs.append(
            count(
                when(col(c).isNull() | isnan(col(c)), c)
            ).alias(c)
        )
    # Non-numeric → chỉ check NULL
    else:
        exprs.append(
            count(
                when(col(c).isNull(), c)
            ).alias(c)
        )

missing_df = train_df.select(exprs)
missing_df.show(truncate=False)

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|0          |0       |0     |0   |0  |0  |0    |0    |0     |0   |0    |0       |
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [80]:
cols_to_be_fill = ['Age', 'Cabin', 'Embarked']
train_df.select(*cols_to_be_fill).describe().show()

+-------+------------------+-------+--------+
|summary|               Age|  Cabin|Embarked|
+-------+------------------+-------+--------+
|  count|               746|    746|     746|
|   mean|29.328646112600534|   NULL|    NULL|
| stddev|13.167380905460854|   NULL|    NULL|
|    min|              0.42|    A14|       C|
|    max|              80.0|Unknown|       S|
+-------+------------------+-------+--------+



In [98]:
age_median = train_df.approxQuantile("Age", [0.5], 0.01)[0]
train_df = train_df.fillna({"Age": age_median})
train_df = train_df.fillna({"Cabin": "Unknown"})
train_df = train_df.fillna({"Embarked": "S"})
train_df = train_df.withColumnRenamed("Survived", "label")


In [99]:
test_df = test_df.fillna({"Age": age_median})
test_df = test_df.fillna({"Cabin": "Unknown"})
test_df = test_df.fillna({"Embarked": "S"})
test_df = test_df.withColumnRenamed("Survived", "label")


In [100]:
from pyspark.ml.feature import StringIndexer

sex_idx = StringIndexer(
    inputCol="Sex", outputCol="SexIdx", handleInvalid="keep"
)

emb_idx = StringIndexer(
    inputCol="Embarked", outputCol="EmbarkedIdx", handleInvalid="keep"
)

cabin_idx = StringIndexer(
    inputCol="Cabin", outputCol="CabinIdx", handleInvalid="keep"
)


In [101]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(
    inputCols=["SexIdx", "EmbarkedIdx", "CabinIdx"],
    outputCols=["SexVec", "EmbarkedVec", "CabinVec"]
)


In [102]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=[
        "Pclass", "Age", "SibSp", "Parch", "Fare",
        "SexVec", "EmbarkedVec", "CabinVec"
    ],
    outputCol="features"
)


In [103]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(
    featuresCol="features",
    labelCol="label",
    maxIter=50,
    maxDepth=5,
    stepSize=0.1
)


In [104]:
from pyspark.ml import Pipeline

gbt_pipeline = Pipeline(stages=[
    sex_idx,
    emb_idx,
    cabin_idx,
    encoder,
    assembler,
    gbt
])

gbt_model = gbt_pipeline.fit(train_df)
gbt_pred = gbt_model.transform(test_df)


In [105]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

print("GBT AUC:", evaluator.evaluate(gbt_pred))


GBT AUC: 0.9122655951014166


### Exercise

Do the classification in the case of data Titanic for other ML model in Spark.
```python
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import OneVsRest
```

In [106]:
from pyspark.ml import Pipeline

lr_pipeline = Pipeline(stages=[
    sex_idx,
    emb_idx,
    cabin_idx,
    encoder,
    assembler,
    lr
])

t_lr_model = lr_pipeline.fit(train_df)
t_lr_pred = t_lr_model.transform(test_df)


In [125]:
feature_pipeline = Pipeline(stages=[
    sex_idx,
    emb_idx,
    cabin_idx,
    encoder,
    assembler
])

feature_model = feature_pipeline.fit(train_df)
train_fe = feature_model.transform(train_df)

num_features = train_fe.select("features").first()["features"].size
print(num_features)

136


In [126]:
lr = LogisticRegression(
    featuresCol="features",
    labelCol="label"
)

dt = DecisionTreeClassifier(
    featuresCol="features",
    labelCol="label",
    maxDepth=6
)

rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    numTrees=100,
    maxDepth=8,
    seed=42
)

gbt = GBTClassifier(
    featuresCol="features",
    labelCol="label",
    maxIter=50,
    maxDepth=5
)

nb = NaiveBayes(
    featuresCol="features",
    labelCol="label"
)

mlp = MultilayerPerceptronClassifier(
    featuresCol="features",
    labelCol="label",
    layers=[num_features, 32, 16, 2],
    maxIter=100
)

In [127]:
dt_pipeline = Pipeline(stages=[
    sex_idx,
    emb_idx,
    cabin_idx,
    encoder,
    assembler, dt])
rf_pipeline = Pipeline(stages=[
    sex_idx,
    emb_idx,
    cabin_idx,
    encoder,
    assembler, rf])
nb_pipeline = Pipeline(stages=[
    sex_idx,
    emb_idx,
    cabin_idx,
    encoder,
    assembler, nb])
mlp_pipeline = Pipeline(stages=[
    sex_idx,
    emb_idx,
    cabin_idx,
    encoder,
    assembler, mlp])
ovr_pipeline = Pipeline(stages=[
    sex_idx,
    emb_idx,
    cabin_idx,
    encoder,
    assembler, ovr])


In [128]:
t_dt_model = dt_pipeline.fit(train_df)
t_dt_pred = t_dt_model.transform(test_df)

t_rf_model = rf_pipeline.fit(train_df)
t_rf_pred = t_rf_model.transform(test_df)

t_nb_model = nb_pipeline.fit(train_df)
t_nb_pred = t_nb_model.transform(test_df)

t_mlp_model = mlp_pipeline.fit(train_df)
t_mlp_pred = t_mlp_model.transform(test_df)

t_ovr_model = ovr_pipeline.fit(train_df)
t_ovr_pred = t_ovr_model.transform(test_df)

In [129]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

auc = evaluator.evaluate(t_dt_pred)
print("Decision Tree AUC:", auc)

auc = evaluator.evaluate(t_rf_pred)
print("Random Forest AUC:", auc)

auc = evaluator.evaluate(t_nb_pred)
print("Naive Bayes AUC:", auc)

auc = evaluator.evaluate(t_mlp_pred)
print("Multilayer Perceptron AUC:", auc)

auc = evaluator.evaluate(t_ovr_pred)
print("One-vs-Rest AUC:", auc)



Decision Tree AUC: 0.6623612705702258
Random Forest AUC: 0.8932261768082663
Naive Bayes AUC: 0.4701492537313433
Multilayer Perceptron AUC: 0.8928434749330273
One-vs-Rest AUC: 0.863758132414849


### The other approaches will be taken as exercises